### Problem formulation

some text here
<br/><br/><br/>

In [233]:
import findspark
findspark.init()

### Import all needed libs

In [234]:
# basically spark
import pyspark
import operator
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row

# processing
import re
from datetime import datetime

# text preprocessing
import nltk
from nltk.corpus import stopwords

### Global variables definition

**User-specific variables**  
Please feel free to tweak those variables as you wish. For example, you can set number of last hours to get hottest topics.

In [221]:
number_of_hours_to_get_topics = 2
frame_start_datetime = "Sun Jun 27 00:00:00 +0000 2019"
frame_finish_datetime = "Sun Jun 30 23:00:00 +0000 2019"

**Technical variables**  
Those variables are needed to connect to db and other technical stuff.

In [ ]:
tweets_table = "usa_training_tweets_30_06.training_tweets_collection"

### Handy functions

In [6]:
### we should put this block in the util functions file

# def str_to_tweeter_datetime():
#     ts2 = datetime.strptime(frame_start_datetime,'%a %b %d %H:%M:%S %z %Y')
#     ts = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(frame_start_datetime,'%a %b %d %H:%M:%S +0000 %Y'))

In [7]:
def str_tweet_to_datetime(frame_datetime):
    ts = datetime.strptime(frame_datetime,'%a %b %d %H:%M:%S %z %Y')
    return ts

def datetime_to_tweet_str(frame_datetime):
    #print(type(frame_datetime))
    ts = datetime.strftime(frame_datetime, '%a %b %d %H:%M:%S %z %Y')
    return ts

def tweet2google_timeframe(frame_start_datetime, frame_finish_datetime):
    start_date = str_tweet_to_datetime(frame_start_datetime)
    end_date = str_tweet_to_datetime(frame_finish_datetime)
    tim

In [200]:
#TODO: move this function to utils
def str_rising_to_float(str):
    if str is None:
        return 0.0
    if str == '':
        return 0.0
    if str == 'Breakout':
        return 0.0
    
    str_value = str.split('%')[0]
    if '+' in str_value:
        str_value = str_value.split('+')[1]
        
    if ',' in str_value:
        str_value = str_value.replace(',', '.')
        value = 1000* float(str_value)
        return value
    return float(str_value)

In [213]:
#TODO: move this function to utils
def unique_google_trends_by_time_frame(df):
    data = df.collect()
    rising_dict = {}
    top_dict = {}
    
    geo = data[0]['geo']
    columns = df.columns


    
    for i in range(0, len(data)):
        rising_val = data[i][columns[1]]
        top_value = data[i][columns[2]]
        
        if rising_val in rising_dict:
            rising_dict[rising_val][0] += str_rising_to_float(data[i][columns[3]])
            rising_dict[rising_val][1] += 1
        else:
            rising_dict[rising_val] = [str_rising_to_float(data[i][columns[3]]), 1]
            
        if top_value in top_dict:
            top_dict[top_value][0] += float(data[i][columns[4]])
            top_dict[top_value][1] += 1
        else:
            top_dict[top_value] = [float(data[i][columns[4]]), 1]
    
    
    for key in top_dict:
        top_dict[key] = round(top_dict[key][0] / top_dict[key][1])
        
    for key in rising_dict:
        rising_dict[key] = round(rising_dict[key][0] / rising_dict[key][1])
    
    top_dict = sorted(top_dict.items(), key=operator.itemgetter(1), reverse=True)
    rising_dict = sorted(rising_dict.items(), key=operator.itemgetter(1), reverse=True)
    
    
    seq = []
    len_top = len(top_dict)
    len_rising = len(rising_dict)
    length = max(len_top, len_rising)
    
    row = Row(columns[1], columns[2], columns[3], columns[4], columns[5])
    
    for i in range(0, length):
        rising = rising_dict[i][0] if i < len_rising else ''
        rising_val = f"+{rising_dict[i][1]}%" if i < len_rising else None
        
        top = top_dict[i][0] if i < len_top else ''
        top_val = top_dict[i][1] if i < len_top else None
        
        seq.append(row(rising, top, rising_val, top_val, geo))
    
    dframe = spark.createDataFrame(seq)
    return dframe

In [8]:
dtf = str_tweet_to_datetime(frame_finish_datetime)

dtf

datetime.datetime(2019, 6, 30, 23, 0, tzinfo=datetime.timezone.utc)

In [9]:
datetime_to_tweet_str(dtf)

'Sun Jun 30 23:00:00 +0000 2019'

### Reading CSV

In [236]:
# df = spark.read.load('./get-tweets-by-geolocation/data/usa_training_tweets_30_06.csv')
df = spark.read.csv('./get-tweets-by-geolocation/training_tweets.csv', inferSchema=True, header=True)

In [231]:
google_trends_search_queries_us = spark.read.csv('data/google-trends/google-trends-search-queries-US.csv', inferSchema=True, header=True)
google_trends_search_topics_us = spark.read.csv('data/google-trends/google-trends-search-topics-US.csv', inferSchema=True, header=True)
google_trends_search_queries_us_ny = spark.read.csv('data/google-trends/google-trends-search-queries-US-NY.csv', inferSchema=True, header=True)
google_trends_search_topics_us_ny = spark.read.csv('data/google-trends/google-trends-search-topics-US-NY.csv', inferSchema=True, header=True)

In [26]:
#TODO: move this function to Handy function block 
def get_google_trends_by_geo(geo):
    if geo == 'US':
        return google_trends_search_topics_us, google_trends_search_queries_us
    elif geo == 'US-NY':
        return google_trends_search_topics_us_ny, google_trends_search_queries_us_ny
    
    return None, None

### Connecting to MongoDB

**Creating Spark connector**

In [227]:
spark = SparkSession.builder.appName("pipeline") \
    .config('spark.mongodb.input.uri', 'mongodb://localhost:27017/'+tweets_table) \
    .config('spark.mongodb.output.uri', 'mongodb://localhost:27017/'+tweets_table) \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.1') \
    .config('spark.mongodb.input.partitioner', 'MongoPaginateBySizePartitioner') \
    .getOrCreate()

## Huge bottleneck, isn't it?

**Creating Spark dataframe on top of DB**

In [69]:
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

In [70]:
# df.createOrReplaceTempView("temp")

In [71]:
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- contributors: null (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- type: string (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |-- dis

In [57]:
# Sample query
# sample = spark.sql('SELECT * FROM temp LIMIT 10')
# sample.collect()

# Data description (Женя)
In order to collect data in a natural way:
<br>- we registered Twitter Developer account;
<br>- using credentials from Twitter Developer account we run script that collected tweets by the geolocation and saved them in mongodb;
<br>
<br><b>As a result:</b>
<br>- we collected  332548 tweets (10Gb in mongodb, ~100Mb in csv) from New-York geolocation since 30 of May up to 15 of June;
<br>- we collected  6617029 tweets (~1.69Gb in csv) from USA geolocation since 15 of June up to now.

### Data preprocessing

#### Selecting data from {{ frame_start_datetime }} to {{frame_finish_datetime}}

In [11]:
sample = spark.sql('SELECT * FROM root LIMIT 10')

AnalysisException: 'Table or view not found: root; line 1 pos 14'

In [59]:
StopWords = stopwords.words("english")

tokens = df.select('text').rdd \
    .map(lambda x: x[0]) \
    .map( lambda document: re.split(" ", document))          \
    .map( lambda word: [x for x in word if x.isalpha()])           \
    .map( lambda word: [x for x in word if len(x) > 3] )           \
    .map( lambda word: [x for x in word if x not in StopWords])    \


AnalysisException: "cannot resolve '`text`' given input columns: [];;\n'Project ['text]\n+- Relation[] MongoRelation(MongoRDD[14] at RDD at MongoRDD.scala:51,Some(StructType()))\n"

In [60]:
tokens.collect()

NameError: name 'tokens' is not defined

### Topic modeling (LDA)

### Hot topics in the USA from [Google trends](https://trends.google.com/trends/explore?geo=US)

Set window time for interesting

In [43]:
start_date = str_tweet_to_datetime(frame_start_datetime)
finish_date = str_tweet_to_datetime(frame_finish_datetime)

In [44]:
geo = 'US-NY'

In [45]:
google_trends_topics, google_trends_queries = get_google_trends_by_geo(geo) 

Google trends topics

In [46]:
interesing_google_topics = google_trends_topics.filter(
    (google_trends_topics.Date >= start_date) & (google_trends_topics.Date <= finish_date))

In [52]:
interesing_google_topics.show(45, True)

+-------------------+----------------------+--------------------+-------+---+-----+
|               Date|Search topics - rising| Search topics - top| Rising|Top|  geo|
+-------------------+----------------------+--------------------+-------+---+-----+
|2019-06-27 00:00:00|  Software - Softwa...|New York - City i...|  +250%|100|US-NY|
|2019-06-27 00:00:00|  Academic degree -...| New York - US State|  +250%| 94|US-NY|
|2019-06-27 00:00:00|          Loan - Topic|        2019 - Topic|  +170%| 41|US-NY|
|2019-06-27 00:00:00|    Management - Topic|Hotel - Building ...|  +170%| 39|US-NY|
|2019-06-27 00:00:00|           Bra - Dress|     Weather - Topic|  +150%| 28|US-NY|
|2019-06-27 00:00:00|  Nike - Footwear m...|Google Search - T...|  +130%| 28|US-NY|
|2019-06-27 00:00:00|  2016 Democratic P...|Google - Technolo...|  +120%| 28|US-NY|
|2019-06-27 00:00:00|  Democratic Party ...|Facebook, Inc. - ...|  +120%| 25|US-NY|
|2019-06-27 00:00:00|  Mortgage loan - T...|Facebook - Social...|  +110%| 25

In case when timeframe is more than 1 day, filter correctly this google-trends

In [218]:
interesing_google_topics_unique= unique_google_trends_by_time_frame(interesing_google_topics)
interesing_google_topics_unique.show()

+----------------------+--------------------+------+---+-----+
|Search topics - rising| Search topics - top|Rising|Top|  geo|
+----------------------+--------------------+------+---+-----+
|  Kamala Harris - U...|New York - City i...|+3550%|100|US-NY|
|  Marianne Williams...| New York - US State|+2200%| 90|US-NY|
|  Brooklyn Nets - B...|        2019 - Topic|+1050%| 42|US-NY|
|  United States wom...|     Weather - Topic| +750%| 37|US-NY|
|  Pete Buttigieg - ...|Hotel - Building ...| +650%| 28|US-NY|
|  Joe Biden - Forme...|Google Search - T...| +600%| 24|US-NY|
|  Kevin Durant - Am...|Google - Technolo...| +600%| 24|US-NY|
|  Boston Red Sox - ...|YouTube - Video s...| +400%| 24|US-NY|
|         Radar - Topic|        Film - Topic| +350%| 24|US-NY|
|    Free agent - Topic|Facebook, Inc. - ...| +350%| 23|US-NY|
|  Software - Softwa...|Facebook - Social...| +250%| 23|US-NY|
|  Academic degree -...|       Sales - Topic| +250%| 20|US-NY|
|  FIFA Women's Worl...|United States - C...| +250%| 20

Google trends queries

In [48]:
interesing_google_queries = google_trends_queries.filter(
    (google_trends_queries.Date >= start_date) & (google_trends_queries.Date <= finish_date))

In [49]:
interesing_google_queries.show()

+-------------------+-----------------------+--------------------+-------+---+-----+
|               Date|Search queries - rising|Search queries - top| Rising|Top|  geo|
+-------------------+-----------------------+--------------------+-------+---+-----+
|2019-06-27 00:00:00|         gerrymandering|                 you|+4,250%|100|US-NY|
|2019-06-27 00:00:00|          tulsi gabbard|             weather|+1,850%| 74|US-NY|
|2019-06-27 00:00:00|            cory booker|              google|+1,450%| 73|US-NY|
|2019-06-27 00:00:00|          julian castro|            facebook|+1,200%| 62|US-NY|
|2019-06-27 00:00:00|   india vs west ind...|                news|  +750%| 48|US-NY|
|2019-06-27 00:00:00|           john delaney|              amazon|  +450%| 47|US-NY|
|2019-06-27 00:00:00|       elizabeth warren|             youtube|  +400%| 45|US-NY|
|2019-06-27 00:00:00|              ind vs wi|              debate|  +250%| 36|US-NY|
|2019-06-27 00:00:00|              powerball|           world cup

In [217]:
interesing_google_queries_unique= unique_google_trends_by_time_frame(interesing_google_queries)
interesing_google_queries_unique.show()

+-----------------------+--------------------+------+---+-----+
|Search queries - rising|Search queries - top|Rising|Top|  geo|
+-----------------------+--------------------+------+---+-----+
|                     yy|                 you|+4950%|100|US-NY|
|         gerrymandering|             weather|+4250%| 94|US-NY|
|          shay mitchell|               pride|+3700%| 62|US-NY|
|        darren collison|              google|+2950%| 60|US-NY|
|    marianne williamson|            facebook|+2750%| 60|US-NY|
|          kamala harris|             youtube|+2400%| 48|US-NY|
|         deandre jordan|                news|+2400%| 45|US-NY|
|          tulsi gabbard|              amazon|+1850%| 45|US-NY|
|            cory booker|           world cup|+1450%| 38|US-NY|
|          julian castro|              debate|+1200%| 36|US-NY|
|       india vs england|             yankees|+1050%| 34|US-NY|
|   argentina vs vene...|        pride parade| +900%| 30|US-NY|
|             usa france|               

#### Hot topics - google trends (directly) (probably this will be removed)

In [118]:
!ls

data			   README.md
get-google-trends-data	   realtime_twitter_topic_modeling_pyspark.ipynb
get-tweets-by-geolocation  sample_lda_data.txt
LICENSE			   spark_LDA.ipynb
python-data-preprocessing  spark_LDA_v2.ipynb
python_LDA.ipynb	   spark-warehouse


In [128]:
from get_google_trends_data.pytrends.pytrends.request import TrendReq
pytrend = TrendReq()
timeframe = "2019-06-5 2019-06-20"
pytrend.build_payload(kw_list=[' '], geo='US', timeframe=timeframe)
topics_df = pytrend.related_top_topics_by_top()

In [129]:
topics_df

,Search topics,Top
0,2019 - Topic,100
1,Weather - Topic,85
2,Facebook - Social networking service,73
3,"Facebook, Inc. - Social network company",73
4,Google Search - Topic,69
5,Google - Technology company,67
6,YouTube - Video sharing company,64
7,Film - Topic,60
8,Car - Transportation mode,47
9,Texas - US State,46


### Conclusion